In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_32.csv')
pred2 = dd.read_csv('test_pred_33.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 32.6 s, sys: 3 s, total: 35.6 s
Wall time: 7.16 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 205 ms, sys: 248 ms, total: 452 ms
Wall time: 450 ms


In [5]:
pred1['object_id'].equals(pred2['object_id'])

True

In [6]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 276 µs, sys: 35 µs, total: 311 µs
Wall time: 315 µs


In [7]:
coef_lgb = 0.8
coef_mlp = 0.2

In [8]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 265 ms, sys: 595 ms, total: 859 ms
Wall time: 857 ms


In [9]:
print(test_ensemble.shape)

(3492890, 15)


In [10]:
test_ensemble['object_id'] = pred1['object_id']
print(test_ensemble.shape)

(3492890, 16)


In [11]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [12]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000228,0.353038,0.000424,0.195240,0.137085,0.000486,0.099774,0.000261,0.002038,0.016563,0.002586,0.191285,0.000327,0.000665,0.245549
3492886,130787971,0.000237,0.011051,0.000257,0.047418,0.244342,0.000318,0.018024,0.310791,0.000425,0.022202,0.000997,0.339970,0.000301,0.003665,0.243434
3492887,130787974,0.000639,0.017662,0.000165,0.890689,0.021799,0.000384,0.054280,0.000067,0.000199,0.001176,0.000421,0.010705,0.000263,0.001549,0.080870
3492888,130788053,0.000092,0.711806,0.000147,0.133919,0.004172,0.000175,0.009691,0.103313,0.000512,0.000507,0.000317,0.034184,0.000151,0.001013,0.163877
3492889,130788054,0.000275,0.098496,0.000372,0.457195,0.119971,0.000783,0.053355,0.000475,0.001065,0.021815,0.002077,0.242366,0.000808,0.000948,0.226374


In [13]:
%%time
test_ensemble.to_csv('test_pred_34.csv',index=False)

CPU times: user 1min 31s, sys: 1.14 s, total: 1min 32s
Wall time: 1min 33s


In [14]:
test_ensemble['object_id'].equals(pred1['object_id'])

True

In [15]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_34.csv -m "Message"

100%|█████████████████████████████████████| 1.06G/1.06G [2:09:45<00:00, 146kB/s]
Successfully submitted to PLAsTiCC Astronomical Classification